In [1]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import math
import torch
from torch import nn
from torch import optim
from tensorboardX import SummaryWriter
from datetime import datetime
import time
import pygame
import load
from models import DynamicTopologyModel
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
EXPERIMENT=1
writer = SummaryWriter('runs/experiment_{}/'.format(EXPERIMENT))

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def save_parameters(writer, model, batch_idx):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("{}_{}".format(k, shape), v.data, batch_idx)

In [3]:
def test_parameters_update(last_parameters_np ,parameters_np):
    for last_param, param in zip(last_parameters_np, parameters_np):
        diff = param - last_param
        #print(last_parameters.max())
        print(diff.max())
        #print(not np.all(diff))


In [4]:
def print_example(reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
                  reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
                  neighbourhood_cell_present_target):
    print("reference_cell_input") 
    print(reference_cell_input)
    print("reference_cell_present_input") 
    print(reference_cell_present_input)
    print("neighbourhood_cell_rel_input") 
    print(neighbourhood_cell_rel_input)
    print("neighbourhood_cell_load_input") 
    print(neighbourhood_cell_load_input)
    print("neighbourhood_cell_present_input") 
    print(neighbourhood_cell_present_input)
    print("reference_cell_target") 
    print(reference_cell_target)
    print("reference_cell_present_target") 
    print(reference_cell_present_target)
    print("neighbourhood_cell_rel_target") 
    print(neighbourhood_cell_rel_target)
    print("neighbourhood_cell_present_target") 
    print(neighbourhood_cell_present_target)

In [5]:
BATCH_SIZE = 32 
INPUT_SEQ_LEN = 2
TARGET_SEQ_LEN = 2
LEARNING_RATE = 1e-3
SAVE_VARIABLE_EVERY = 100
DATASET_WORKERS = 4


input_seq_len= torch.Tensor([INPUT_SEQ_LEN]).int()
target_seq_len= torch.Tensor([TARGET_SEQ_LEN]).int()

# Define Data.
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.5, 0.5
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=DATASET_WORKERS)

# Define Model
dynamic_topology_model = DynamicTopologyModel(neighbourhood_hidden_size=32, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, lstm_hidden_size=32, lstm_layers=2, teacher_forcing_probability=0.5, device=device).to(device)

parameters = dynamic_topology_model.parameters

# Define optimizer
optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
last_parameters_np = list(np.copy(param.cpu().detach().numpy()) for param in parameters)

# Train
start = datetime.now()
for batch_idx, data in enumerate(dataloader):
    optimizer.zero_grad()
    reference_cell_input =  data[0].to(device)
    reference_cell_present_input = data[1].to(device)
    neighbourhood_cell_rel_input = data[2].to(device)
    neighbourhood_cell_load_input = data[3].to(device)
    neighbourhood_cell_present_input = data[4].to(device)
    reference_cell_target = data[5].to(device)
    reference_cell_present_target = data[6].to(device)
    neighbourhood_cell_rel_target = data[7].to(device)
    neighbourhood_cell_present_target = data[8].to(device)
    
    #print_example(reference_cell_input, reference_cell_present_input,
    #            neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
    #              reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
    #              neighbourhood_cell_present_target)    
    
    decoder_output_seq = dynamic_topology_model.forward(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
    
    loss = criterion(decoder_output_seq, reference_cell_target)
    loss.backward()
    optimizer.step()
    if batch_idx == 0:
        parameters = dynamic_topology_model.parameters
        parameters_np = list(param.cpu().detach().numpy() for param in parameters)
        test_parameters_update(last_parameters_np, parameters_np)
        inputs =(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
        writer.add_graph(dynamic_topology_model, inputs, True)
    if batch_idx % SAVE_VARIABLE_EVERY == 0:
        end = datetime.now()
        print("{} Examples/sec".format((SAVE_VARIABLE_EVERY* BATCH_SIZE) / ((end - start).total_seconds()) ))
        start = datetime.now()
        print("loss: {}".format(loss))
        #print("output: {}".format(decoder_output_seq))
        #print("target: {}".format(reference_cell_target))
        writer.add_scalar("loss", loss, batch_idx)
        save_parameters(writer, dynamic_topology_model, batch_idx)


0.00094366074
0.0009949803
0.0009996295
0.0009998679
0.00089740753
-0.0009992644
0.0009987215
-0.0009999871
0.0009999834
0.0
0.0009999722
0.0009999722
0.0009999573
0.0
0.0009999946
0.0009999946
0.0
0.0
0.0009999275
0.0009999871
0.001000002
0.0009999946
0.001000002
0.0009999424
0.000999989
0.0009999871
0.0009999946
0.0009999657
0.001000002
0.001000002
0.001000002
0.0009999999
graph(%0 : Int(1)
      %1 : Int(1)
      %2 : Float(32, 2, 1)
      %3 : Float(32, 2, 1)
      %4 : Float(32, 2, 24, 2)
      %5 : Float(32, 2, 24, 1)
      %6 : Float(32, 2, 1)
      %7 : Float(32, 2, 1)
      %8 : Float(32, 2, 24, 2)
      %9 : Float(32, 2)
      %10 : Float(32)
      %11 : Float(32, 1)
      %12 : Float(32)
      %13 : Float(1, 32)
      %14 : Float(1)
      %15 : Float(1, 32)
      %16 : Float(1)
      %17 : Float(128, 3)
      %18 : Float(128, 32)
      %19 : Float(128)
      %20 : Float(128)
      %21 : Float(128, 32)
      %22 : Float(128, 32)
      %23 : Float(128)
      %24 : Float(128)
 

/Users/d854956/personal/dynamic_topology/models.py:196: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  (batch_size, target_seq_len.data.numpy()[0], 1), device=self.device
/Users/d854956/personal/dynamic_topology/models.py:200: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for input_seq_idx in range(input_seq_len):
/Users/d854956/personal/dynamic_topology/models.py:231: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the f

2017.611223971239 Examples/sec
loss: 1.1644065380096436
2094.003754810483 Examples/sec
loss: 0.32445335388183594
1838.6683444515309 Examples/sec
loss: 0.1151362806558609
2174.8128131815406 Examples/sec
loss: 0.04087473079562187
2161.5049478199194 Examples/sec
loss: 0.039010874927043915
2172.103940603818 Examples/sec
loss: 0.025958949699997902
2065.6449040217462 Examples/sec
loss: 0.013768710196018219
2176.016624767013 Examples/sec
loss: 0.010378601029515266
2273.350506601952 Examples/sec
loss: 0.016534928232431412
2229.211729833471 Examples/sec
loss: 0.027125436812639236
1962.6952958487154 Examples/sec
loss: 0.013313929550349712
2186.642212717921 Examples/sec
loss: 0.0039010958280414343
2287.485453022197 Examples/sec
loss: 0.2142162173986435
1899.9543417222255 Examples/sec
loss: 0.005800675600767136
1834.4458483050867 Examples/sec
loss: 0.014105859212577343
2184.44348571647 Examples/sec
loss: 0.012093869969248772
2175.2134428190766 Examples/sec
loss: 0.031644899398088455
2190.294122277

KeyboardInterrupt: 

In [6]:
PIXELS = 640
LEFT = 1
RIGHT = 3
INPUT_SCREEN_COUNT = 2
TARGET_SCREEN_COUNT = 2
SCREEN_COUNT = INPUT_SCREEN_COUNT + TARGET_SCREEN_COUNT

PIXELS_PER_BLOCK = int(PIXELS / IMAGE_SIZE)
SCALE_ARR = np.ones((PIXELS_PER_BLOCK, PIXELS_PER_BLOCK, 1))

load_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 1))
load_screen[1,12, 0] = 255
load_screen[22,22, 0] = 255
loads = [(1, 12), (22, 22)]
block_img = np.kron(load_screen, SCALE_ARR)

screen = pygame.display.set_mode((PIXELS, PIXELS))
screens = []

screen_draw_idx = 0
draw_on = False
last_pos = (0, 0)
color = (255, 128, 0)
radius = 10
block_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))
model_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
frame = 0

In [7]:
def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells


In [16]:
 0
cells_seq = [list() for i in range(SCREEN_COUNT)]
cells_load_seq = [list() for i in range(SCREEN_COUNT)]
BATCH_SIZE=1
show_labels = True
input_seq_len= torch.Tensor([INPUT_SCREEN_COUNT]).int()
target_seq_len= torch.Tensor([TARGET_SCREEN_COUNT]).int()

while True:
    
    ##############################
    # Use model to forecast future load
    ##############################
    load_cells_seq_input = cells_load_seq[:INPUT_SCREEN_COUNT]
    load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
    
 
    model_target = [list() for i in range(TARGET_SCREEN_COUNT)]
    
    if show_labels:
        caption = "Label frame {}".format(screen_draw_idx)
    else:
        caption = "Model frame {}".format(screen_draw_idx)
    pygame.display.set_caption(caption)
    for seq_idx  in range(TARGET_SCREEN_COUNT):
        for reference_cell in cells_seq[INPUT_SCREEN_COUNT + seq_idx]:
            #print(reference_cell)
            ref_x, ref_y = reference_cell[0], reference_cell[1]

            # Get inputs from state.
            (reference_cell_input,
             reference_cell_present_input,
             neighbourhood_cell_rel_input,
             neighbourhood_cell_load_input,
             neighbourhood_cell_present_input,
             reference_cell_target,
             reference_cell_present_target,
             neighbourhood_cell_rel_target,
             neighbourhood_cell_present_target,) = load_topology_dataset.LoadCellDataset.build_inputs_and_targets(INPUT_SCREEN_COUNT,
                                                                                                 TARGET_SCREEN_COUNT,
                                                                                                 ref_x, ref_y,
                                                                                                 load_cells_seq_input,
                                                                                                 load_cells_seq_target)
            # Infer output from model.  Add batch dimension to each input since dataloader is not used here.
            decoder_output_seq = dynamic_topology_model.forward(input_seq_len, target_seq_len,
                                                                reference_cell_input.unsqueeze(0),
                                                                reference_cell_present_input.unsqueeze(0),
                                                                neighbourhood_cell_rel_input.unsqueeze(0),
                                                                neighbourhood_cell_load_input.unsqueeze(0),
                                                                reference_cell_target.unsqueeze(0),
                                                                reference_cell_present_target.unsqueeze(0),
                                                                neighbourhood_cell_rel_target.unsqueeze(0))

            model_target[seq_idx].append((ref_x, ref_y, decoder_output_seq.detach().numpy()[0][seq_idx][0]))    
    ################################
    # DISPLAY
    #################################
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    cell_loads = cells_load_seq[screen_draw_idx]

    if (screen_draw_idx < INPUT_SCREEN_COUNT) or (show_labels and screen_draw_idx >= INPUT_SCREEN_COUNT):
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
    else:
        model_loads = model_target[screen_draw_idx - INPUT_SCREEN_COUNT]
        for cell in model_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
        
        # Update all cell loads
        for i, cells in enumerate(cells_seq):
            cells_load_seq[i] = load.calculate_cell_load(cells, loads)
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if show_labels:
                show_labels = False
            else:
                show_labels = True
                print(model_target[-1])
                print(load_cells_seq_target[-1])
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))

[(7, 13, 0.2618928), (16, 13, 0.27077663), (22, 8, 0.26640362)]
[(7, 13, 0.6860537046677078), (16, 13, 0.685416716720725), (22, 8, 0.6285295786115672)]
[(7, 13, 0.3212703), (16, 13, 0.32657582), (22, 8, 0.32614085)]
[(7, 13, 0.6860537046677078), (16, 13, 0.685416716720725), (22, 8, 0.6285295786115672)]
[(7, 13, 0.31517512), (16, 13, 0.32657582), (22, 8, 0.32614085)]
[(7, 13, 0.6860537046677078), (16, 13, 0.685416716720725), (22, 8, 0.6285295786115672)]
[(7, 13, 0.17725854), (16, 13, 0.17838275), (22, 8, 0.16855527), (7, 9, 0.24266964), (25, 4, 0.17919074), (27, 2, 0.18530464)]
[(7, 13, 0.3646075985716737), (16, 13, 0.3638785918691302), (22, 8, 0.3335114973407248), (7, 9, 0.3554486627957718), (25, 4, 0.3023787058769728), (27, 2, 0.2801749435457266)]
[(7, 13, 0.13601436), (16, 13, 0.13705054), (22, 8, 0.1348989), (7, 9, 0.18295646), (25, 4, 0.14038509), (27, 2, 0.15011573), (10, 22, 0.1638842)]
[(7, 13, 0.30819357595053354), (16, 13, 0.3076255430226834), (22, 8, 0.28197354768356153), (7,

KeyboardInterrupt: 

In [14]:
for seq_idx  in range(TARGET_SCREEN_COUNT):
    print(seq_idx)
    model_target[seq_idx].append((ref_x, ref_y, decoder_output_seq.detach().numpy()[0][seq_idx][0]))

0
1


In [12]:
decoder_output.detach().numpy()[seq_idx][0]

NameError: name 'decoder_output' is not defined